## Udacity P3: Behavior Cloning

In [1]:
from library import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.python.control_flow_ops = tf
dropout_rate = 0.25
learning_rate = 1e-03

# Number of neurons
neuron_100 = 100
neuron_50 = 50
neuron_10 = 10
neuron_1 = 1

# Number of frames in each convolution layers
conv_layer_1 = 24
conv_layer_2 = 36
conv_layer_3 = 48
conv_layer_4 = 64

# Number of epochs
epoch_no = 5
pooling_size = (2,2) 
activation_func = 'relu'
loss_type = 'mean_squared_error'
batch_size = 32


Using TensorFlow backend.


In [2]:
#  generator function to augment and tune train data
from train_generator_lib import train_generator
#  generator function to tune validation data
from valid_generator_lib import valid_generator


In [3]:
from sklearn.utils import shuffle
from keras.models import model_from_json
# images of size are 320x160
local_data_path = "C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/data"
os.chdir(r"C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/data")
#os.chdir(r"C:\Users\NIKHIL XAVIER\git\Self-Driving-Car\P3\CarND-Behavioral-Cloning-P3-master\data")

new_data_path = os.path.join(local_data_path, "IMG/", "*.jpg")
final_list = []
turn_str =[]
turn_lft = []
turn_rgt = []

df = pd.io.parsers.read_csv(os.path.join(local_data_path, 'driving_log.csv'))
# Use as dataframe
df = pd.read_csv('driving_log.csv', header=0)
df.columns = ["center", "left","right", "steering", "throttle", "break", "speed"]
df.drop(['throttle', 'break', 'speed'], axis = 1, inplace = True)
    
for counter in range(len(df)):
    keep_prob = random.random()
        if (df["steering"][counter] >0.20 and df["steering"][counter] <=0.50):
            new_steering = df["steering"][counter]*(1.0 + np.random.uniform(-1,1)/100.0)
            turn_rgt.append([df["center"][counter], df["left"][counter], df["right"][counter], new_steering])
            new_steering = df["steering"][counter]*(1.0 + np.random.uniform(-1,1)/100.0)
            turn_rgt.append([df["center"][counter], df["left"][counter], df["right"][counter], new_steering])

        elif (df["steering"][counter] >= -0.50 and df["steering"][counter] < -0.15):
            new_steering = df["steering"][counter]*(1.0 + np.random.uniform(-1,1)/100.0)
            turn_lft.append([df["center"][counter], df["left"][counter], df["right"][counter], new_steering])
            new_steering = df["steering"][counter]*(1.0 + np.random.uniform(-1,1)/100.0)
            turn_lft.append([df["center"][counter], df["left"][counter], df["right"][counter], new_steering])

        elif (df["steering"][counter] > -0.02 and df["steering"][counter] < 0.02):
            if (keep_prob <=0.90):
                turn_str.append([df["center"][counter], df["left"][counter], df["right"][counter], df["steering"][counter]])
            elif (keep_prob >0.90):
                turn_str.append([df["center"][counter], df["left"][counter], df["right"][counter], df["steering"][counter]])

                
    final_list = turn_rgt + turn_lft + turn_str
    print(len(final_list), len(turn_str), len(turn_lft), len(turn_rgt))
    random.shuffle(final_list)
                
# create sets for validation data set and train data set
df_train, df_valid = sklearn.model_selection.train_test_split(final_list, test_size=.20)

# Model architecture
model = models.Sequential()
model.add(layers.core.Lambda(lambda x: (x / 127.5 - 1.), input_shape = (160,320,3)))
model.add(layers.convolutional.Convolution2D(conv_layer_1, 5, 5, activation=activation_func))
model.add(layers.pooling.MaxPooling2D(pool_size=pooling_size))
model.add(layers.convolutional.Convolution2D(conv_layer_2, 5, 5, activation=activation_func))
model.add(layers.pooling.MaxPooling2D(pool_size=pooling_size))
model.add(layers.convolutional.Convolution2D(conv_layer_3, 5, 5, activation=activation_func))
model.add(layers.pooling.MaxPooling2D(pool_size=pooling_size))
model.add(layers.convolutional.Convolution2D(conv_layer_4, 3, 3, activation=activation_func))
model.add(layers.pooling.MaxPooling2D(pool_size=pooling_size))
model.add(layers.core.Flatten())
model.add(layers.core.Dense(neuron_100, activation=activation_func))
model.add(layers.core.Dropout(dropout_rate))
model.add(layers.core.Dense(neuron_50, activation=activation_func))
model.add(layers.core.Dropout(dropout_rate))
model.add(layers.core.Dense(neuron_10, activation=activation_func))
model.add(layers.core.Dense(neuron_1))
model.compile(optimizer=optimizers.Adam(lr=learning_rate), loss=loss_type)
    
nb_epoch = 5
samples_per_epoch = 20000
nb_val_samples = 2000
  
from keras.callbacks import ModelCheckpoint
path_link="C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/checkpoint2/check-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath= path_link, verbose=1, save_best_only=False)
callbacks_list = [checkpoint]
    
train_generator = train_generator(df_train, batch_size=batch_size, key = 1)
validation_generator = valid_generator(df_valid, batch_size=batch_size, key = 0)

history_object = model.fit_generator(train_generator, samples_per_epoch= samples_per_epoch,
                                     validation_data=validation_generator,
                                     nb_val_samples=nb_val_samples, nb_epoch=nb_epoch, verbose=1, callbacks=callbacks_list)

model_json = model.to_json()
with open("C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/model_final.json", "w") as json_file:
    json_file.write(model_json)

model.save("C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/model_final.h5")
print("Saved model to disk")
print(model.summary())

6445 4557 1138 750
Epoch 1/5
19987/20000 [============================>.] - ETA: 3s - loss: 0.0210 

C:\Users\NIKHIL XAVIER\Miniconda3\envs\carnd-term1\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: saving model to C:/Users/NIKHIL XAVIER/git/Self-Driving-Car/P3/CarND-Behavioral-Cloning-P3-master/checkpoint2/check-00-0.0118.hdf5
20043/20000 [==============================] - 5097s - loss: 0.0210 - val_loss: 0.0118
Epoch 2/5
20029/20000 [==============================] - 6355s - loss: 0.0141 - val_loss: 0.0110
Epoch 3/5
20044/20000 [==============================] - 4610s - loss: 0.0129 - val_loss: 0.0126
Epoch 4/5
20051/20000 [==============================] - 6308s - loss: 0.0124 - val_loss: 0.0116
Epoch 5/5
20024/20000 [==============================] - 5492s - loss: 0.0116 - val_loss: 0.0101
Saved model to disk
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
__________________________________________________________